# Figure 2 a

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
from scipy import spatial
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import calc_subspace_proj_error

from matplotlib import cm

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e1/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'

mmap, verbose = True, True

p,T_full,n,snr = 1000, 100000, 20, (9., 9.)
Ts = [1000, 2000, 5000] + [i*10000 for i in range(1,11)]

run = '_e1_init'
file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) + 'snr' + str(np.int(np.mean(snr)//1)) +  run
load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
pars_true = load_file['pars_true'].copy()
idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()

ev_true = np.linalg.eigvals(pars_true['A'])
ev_idx = np.argsort(np.angle(ev_true))
ev_true = ev_true[ev_idx][n//2:]    
std_angl = np.std(np.angle(ev_true))
std_tmsc = np.std(np.log(1-np.abs(ev_true)))    


def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

clrs = cm.jet(np.linspace(50, 200, len(Ts), dtype=int))
lgnd = []


def find_nearest_pairs(x,y): 
    n,d = x.shape
    assert np.all((n,d) == y.shape)

    out = np.zeros((n,2),dtype=int)
    rx, ry = np.arange(n,dtype=int), np.arange(n,dtype=int)
    pwd = spatial.distance.squareform(spatial.distance.pdist(np.vstack((x,y))))[:n,n:]
    for i in range(n):
        out[i,:] = np.unravel_index(np.argmin(pwd[np.ix_(rx,ry)]), dims=(len(rx),len(ry)))
        out[i,:] = [rx[out[i,0]], ry[out[i,1]]]
        rx = np.setdiff1d(rx, out[i,0])        
        ry = np.setdiff1d(ry, out[i,1])     
        
    return out

plt.figure(figsize=(10,4))

subsp_errors   = np.zeros(len(Ts))
dyn_errors_abs = np.zeros(len(Ts))
dyn_errors_agl = np.zeros(len(Ts))

subsp_errors_g   = np.zeros(len(Ts))
dyn_errors_abs_g = np.zeros(len(Ts))
dyn_errors_agl_g = np.zeros(len(Ts))

for Ti in range(len(Ts)):
    T = Ts[Ti]
    print('T = ', str(T))
    
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+'snr'+str(np.int(np.mean(snr)//1))+'_e1_final'
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    obs_scheme = load_file['obs_scheme']
    pars_est = load_file['pars_est']
    traces = load_file['traces']
    W = load_file['W']
    Qs = load_file['Qs']
    Om = load_file['Om']
    
    try: 
        file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+'snr'+str(np.int(np.mean(snr)//1))+'_e1_GROUSE'
        load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
        print('loading GROUSE fit')
    except:
        pass
    pars_est_g = load_file['pars_g']
    traces_g = load_file['traces_g']
    
        
    subsp_errors[Ti] = calc_subspace_proj_error(pars_true['C'], pars_est['C'])
    subsp_errors_g[Ti] = calc_subspace_proj_error(pars_true['C'], pars_est_g['C'])
   
    plt.subplot(1,2,2)

    # ssidid
    ev_est = np.linalg.eigvals(pars_est['A'])
    ev_idx = np.argsort(np.angle(ev_est))
    ev_est = ev_est[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true)/std_angl, np.log(1-np.abs(ev_true))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est )/std_angl, np.log(1-np.abs(ev_est ))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)                
    #ev_true = ev_true[out[:,0]]
    #ev_est = ev_est[out[:,1]]            
    #dyn_errors_abs[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl[Ti] = np.sqrt(np.mean( (np.angle(ev_est) - np.angle(ev_true))**2))/np.pi * 90    
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs[Ti] = kde.mean() 
    dyn_errors_agl[Ti] = np.sum(np.log(kde))    

    # GROUSE
    ev_est_g = np.linalg.eigvals(pars_est_g['A'])
    ev_idx = np.argsort(np.angle(ev_est_g))
    ev_est_g = ev_est_g[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true )/std_angl, np.log(1-np.abs(ev_true ))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est_g)/std_angl, np.log(1-np.abs(ev_est_g))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)            
    #ev_true = ev_true[out[:,0]]
    #ev_est_g = ev_est_g[out[:,1]]            
    #dyn_errors_abs_g[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est_g))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl_g[Ti] = np.sqrt(np.mean( (np.angle(ev_est_g) - np.angle(ev_true))**2))/np.pi * 90
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs_g[Ti] = kde.mean() 
    dyn_errors_agl_g[Ti] = np.sum(np.log(kde))    
    
        
plt.subplot(1,2,1)
plt.loglog(Ts, subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.loglog(Ts, subsp_errors, 'o-', color='g', linewidth=2)
plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('T')
plt.title('subspace estimation')
plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))

plt.subplot(1,2,2)
    
#plt.loglog(Ts, dyn_errors_abs, 'g.-')
#plt.loglog(Ts, dyn_errors_agl, 'gx--')
#plt.loglog(Ts, dyn_errors_abs_g, 'b.-')
#plt.loglog(Ts, dyn_errors_agl_g, 'bx--')
#lgnd = ['log_10 1-\lambda', 'angle(\lambda) [deg]']

plt.semilogx(Ts, dyn_errors_agl_g, 'bo-', linewidth=1.5)
plt.semilogx(Ts, dyn_errors_agl, 'go-', linewidth=2)
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=4,frameon=False)
plt.box('off')
plt.ylabel('eigenvalues of A')
plt.xlabel('T')
plt.title('spectrum of dynamics')
plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-3,1))

plt.savefig(fig_path + 'fig2_A.pdf')
plt.show()


In [ ]:
file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+'snr'+str(np.int(np.mean(snr)//1))+'_e1_final'
load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()

traces = load_file['traces']
traces

In [ ]:
plt.semilogx(np.hstack(traces[0]))
plt.plot()

In [ ]:
for Ti in range(len(Ts)):
    T = Ts[Ti]
    print('T = ', str(T))
    

    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+'snr'+str(np.int(np.mean(snr)//1))+'_e1_final'
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    obs_scheme = load_file['obs_scheme']
    pars_est = load_file['pars_est']
    traces = load_file['traces']
    W = load_file['W']
    Qs = load_file['Qs']
    Om = load_file['Om']
        
    plt.figure(figsize=(20,9))
    plt.semilogy(np.vstack(traces[2]))
    plt.show()

In [ ]:

plt.figure(figsize=(16, 8))

plt.subplot(1,2,1)
plt.loglog(Ts, dyn_errors_abs, 'go-')
plt.loglog(Ts, dyn_errors_abs_g, 'bo-')
plt.box('off')
plt.title('eigenvalues of A')
plt.xlabel('T')
plt.ylabel('avg. kernel density value ( < kde_x(y) > )')


plt.subplot(1,2,2)

plt.semilogx(Ts, dyn_errors_agl_g, 'bo-', linewidth=1.5)
plt.semilogx(Ts, dyn_errors_agl, 'go-', linewidth=2)
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=4,frameon=False)
plt.box('off')
plt.title('eigenvalues of A')
plt.xlabel('T')
plt.ylabel('sum of log kernel density values ( \sum_i log kde_x(y_i) )')
plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-3,1))

plt.show()


In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import calc_subspace_proj_error

from matplotlib import cm

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e1/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'

mmap, verbose = True, True

p,T_full,n,snr = 1000, 100000, 20, (9., 9.)
Ts = [1000] + [i*10000 for i in range(1,11)]

run = '_e1_init'
file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) + 'snr' + str(np.int(np.mean(snr)//1)) +  run
load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
pars_true = load_file['pars_true'].copy()
idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()

ev_true = np.linalg.eigvals(pars_true['A'])
ev_idx = np.argsort(np.angle(ev_true))
ev_true = ev_true[ev_idx][n//2:]    
std_angl = np.std(np.angle(ev_true))
std_tmsc = np.std(np.log(1-np.abs(ev_true)))    

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)
lgnd = []


def find_nearest_pairs(x,y): 
    n,d = x.shape
    assert np.all((n,d) == y.shape)

    out = np.zeros((n,2),dtype=int)
    rx, ry = np.arange(n,dtype=int), np.arange(n,dtype=int)
    pwd = spatial.distance.squareform(spatial.distance.pdist(np.vstack((x,y))))[:n,n:]
    for i in range(n):
        out[i,:] = np.unravel_index(np.argmin(pwd[np.ix_(rx,ry)]), dims=(len(rx),len(ry)))
        out[i,:] = [rx[out[i,0]], ry[out[i,1]]]
        rx = np.setdiff1d(rx, out[i,0])        
        ry = np.setdiff1d(ry, out[i,1])     
        
    return out
        

plt.figure(figsize=(10,4))

subsp_errors = np.zeros(len(Ts))
subsp_errors_g = np.zeros(len(Ts))
dyn_errors_abs = np.zeros(len(Ts))
dyn_errors_agl = np.zeros(len(Ts))

for Ti in range(len(Ts)):
    T = Ts[Ti]
    print('T = ', str(T))
    
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+'snr'+str(np.int(np.mean(snr)//1))+'_e1_final'
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    pars_est = load_file['pars_est']
    
    clrs = cm.hsv(np.linspace(0, 255, 20, dtype=int))    
    ev_est = np.linalg.eigvals(pars_est['A'])
    ev_idx = np.argsort(np.angle(ev_est))
    ev_est = ev_est[ev_idx][n//2:]    

    x_ = np.vstack([np.angle(ev_true), np.log(1-np.abs(ev_true))]).T
    y_ = np.vstack([np.angle(ev_est), np.log(1-np.abs(ev_est))]).T
    out = find_nearest_pairs(x=x_,y=y_)        
    
    plt.figure(figsize=(10,10))
    ev_true = ev_true[out[:,0]]
    ev_est = ev_est[out[:,1]]    
    
    for i in range(len(ev_true)):
        plt.plot(np.real(ev_true[i]), np.imag(ev_true[i]), 'o', color=clrs[i,:], 
                 markersize=10,
                 markeredgewidth=3)
        plt.hold(True)
        plt.plot(np.real(ev_est[i]), np.imag(ev_est[i]), 'x', color=clrs[i,:], 
                 markersize=10,
                 markeredgewidth=3)
    plt.show()    
    
    plt.figure(figsize=(10,6))
    plt.subplot(1,3,1)
    plt.hist(np.abs(ev_est) - np.abs(ev_true), bins=np.linspace(-0.025, 0.025, 10))
    plt.subplot(1,3,2)
    plt.hist(np.angle(ev_est) - np.angle(ev_true), bins=np.linspace(-0.025, 0.025, 10))
    plt.subplot(1,3,3)
    plt.hist(np.abs(ev_est-ev_true), bins=np.linspace(0, 0.025, 20) )
    plt.show()
    dyn_errors_abs[Ti] = np.sqrt(np.mean( (np.abs(ev_est)-np.abs(ev_true))**2 ))
    dyn_errors_agl[Ti] = np.sqrt(np.mean( (np.angle(ev_est)/np.pi - np.angle(ev_true)/np.pi)**2))
    
    print('average norm of eigenvalue error: ', np.mean(np.abs(ev_est-ev_true)))
    
plt.loglog(Ts, dyn_errors_abs, 'o-')
plt.loglog(Ts, dyn_errors_agl, 'x-')
plt.show()


# Fig 3 a - data missing at random

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
from scipy import spatial
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import calc_subspace_proj_error

from matplotlib import cm


p,T,n,snr = 1000, 20020, 10, (1., 1.)
rnd_seeds = range(30,40)
fracs_obs = (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
run = '_e3rnd'

mmap, verbose = True, True

fig_path =  '/home/marcel/Desktop/Projects/Stitching/code/le_stitch/python/figs/'

plt.figure(figsize=(10,4))

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

clrs = cm.jet(np.linspace(50, 200, len(fracs_obs), dtype=int))
lgnd = []

for rnd_seed in rnd_seeds:

    data_path =  data_path = '/media/marcel/636f7b46-1fd1-4600-b69e-86d2ed82002c/stitching/hankel/icml_e3/rnd/seed_' + str(rnd_seed) + '/'


    #file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) +  run
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+str(run)+'_'+str(int(100*1.))
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    pars_true = load_file['pars_true'].copy()

    ev_true = np.linalg.eigvals(pars_true['A'])
    #ev_idx = np.argsort(np.angle(ev_true))
    #ev_true = ev_true[n//2:]
    std_angl = np.std(np.angle(ev_true))
    std_tmsc = np.std(np.log(1-np.abs(ev_true)))    

    subsp_errors   = np.zeros(len(fracs_obs))
    dyn_errors_abs = np.zeros(len(fracs_obs))
    dyn_errors_agl = np.zeros(len(fracs_obs))

    subsp_errors_g   = np.zeros(len(fracs_obs))
    dyn_errors_abs_g = np.zeros(len(fracs_obs))
    dyn_errors_agl_g = np.zeros(len(fracs_obs))

    for i in range(len(fracs_obs)):
        frac_obs = fracs_obs[i]

        print('frac_obs = ', str(frac_obs))

        file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T)+str(run)+'_'+str(int(100*frac_obs))
        load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
        idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()
        obs_scheme = load_file['obs_scheme']
        pars_est = load_file['pars_est']
        pars_est_g = load_file['pars_est_g']
        traces = load_file['traces']
        traces_g = load_file['traces_g']
        W = load_file['W']
        Qs = load_file['Qs']
        Om = load_file['Om']

        C = pars_est['C'].copy()
        C[obs_scheme.sub_pops[0]] *= -1
        if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est['C']):
            pars_est['C'][obs_scheme.sub_pops[0]] *= -1
        del C    
        subsp_errors[i] = calc_subspace_proj_error(pars_true['C'], pars_est['C'])


        C = pars_est_g['C'].copy()
        C[obs_scheme.sub_pops[0]] *= -1
        if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est_g['C']):
            pars_est_g['C'][obs_scheme.sub_pops[0]] *= -1
        del C        
        subsp_errors_g[i] = calc_subspace_proj_error(pars_true['C'], pars_est_g['C'])


        # ssidid
        ev_est = np.linalg.eigvals(pars_est['A'])
        ev_idx = np.argsort(np.angle(ev_est))
        ev_est = ev_est[ev_idx][n//2:]        

        x_ = np.vstack([np.angle(ev_true)/std_angl, np.log(1-np.abs(ev_true))/std_tmsc]).T
        y_ = np.vstack([np.angle(ev_est )/std_angl, np.log(1-np.abs(ev_est ))/std_tmsc]).T

        out = find_nearest_pairs(x=x_,y=y_)                
        ev_true = ev_true[out[:,0]]
        ev_est = ev_est[out[:,1]]            
        dyn_errors_abs[i] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est))-np.log10(1-np.abs(ev_true)))**2 ))
        dyn_errors_agl[i] = np.sqrt(np.mean( (np.angle(ev_est) - np.angle(ev_true))**2))/np.pi * 90    

        #kde = sp.stats.gaussian_kde(x_.T)(y_.T)
        #dyn_errors_abs[i] = kde.mean() 
        #dyn_errors_agl[i] = np.sum(np.log(kde))    

        # GROUSE
        ev_est_g = np.linalg.eigvals(pars_est_g['A'])
        ev_idx = np.argsort(np.angle(ev_est_g))
        ev_est_g = ev_est_g[ev_idx][n//2:]        

        x_ = np.vstack([np.angle(ev_true )/std_angl, np.log(1-np.abs(ev_true ))/std_tmsc]).T
        y_ = np.vstack([np.angle(ev_est_g)/std_angl, np.log(1-np.abs(ev_est_g))/std_tmsc]).T

        out = find_nearest_pairs(x=x_,y=y_)            
        ev_true = ev_true[out[:,0]]
        ev_est_g = ev_est_g[out[:,1]]            
        dyn_errors_abs_g[i] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est_g))-np.log10(1-np.abs(ev_true)))**2 ))
        dyn_errors_agl_g[i] = np.sqrt(np.mean( (np.angle(ev_est_g) - np.angle(ev_true))**2))/np.pi * 90

        #kde = sp.stats.gaussian_kde(x_.T)(y_.T)
        #dyn_errors_abs_g[i] = kde.mean() 
        #dyn_errors_agl_g[i] = np.sum(np.log(kde))    


    plt.subplot(1,2,1)
    plt.semilogy(fracs_obs, subsp_errors_g, 'o-', color='b', linewidth=1.5)
    plt.semilogy(fracs_obs, subsp_errors, 'o-', color='g', linewidth=2)
    plt.box('off')
    lgnd = ['GROUSE', 'ssidid']
    plt.legend(lgnd, loc=1,frameon=False)
    plt.ylabel('subsp. proj. error')
    plt.xlabel('% observed')
    plt.title('subspace estimation')
    #plt.xticks(10**np.arange(3,6))
    #plt.yticks(10**np.arange(-2,1))

    plt.subplot(1,2,2)

    plt.plot(fracs_obs, dyn_errors_agl_g, 'bo-', linewidth=1.5)
    plt.plot(fracs_obs, dyn_errors_agl,   'go-', linewidth=2)
    #plt.loglog(fracs_obs, dyn_errors_abs_g, 'b.-')
    #plt.loglog(fracs_obs, dyn_errors_agl_g, 'bx--')
    lgnd = ['GROUSE', 'ssidid']
    plt.legend(lgnd, loc=1,frameon=False)
    plt.box('off')
    plt.ylabel('eigenvalues of A')
    plt.xlabel('% observed')
    plt.title('spectrum of dynamics')
    #plt.xticks(10**np.arange(3,6))
    #plt.yticks(10**np.arange(-3,1))

plt.savefig(fig_path + 'fig3_A.pdf')
plt.show()
    
    

In [ ]:
plt.imshow(pars_true['A'], interpolation='None')
plt.show()
plt.imshow(pars_est['A'], interpolation='None')
plt.show()
plt.imshow(pars_est_g['A'], interpolation='None')
plt.show()

plt.plot(np.sort(np.abs(np.linalg.eigvals(pars_true['A']))), 'k')
plt.plot(np.sort(np.abs(np.linalg.eigvals(pars_est_g['A']))), 'b')
plt.plot(np.sort(np.abs(np.linalg.eigvals(pars_est['A']))), 'g')
plt.show()

plt.plot(np.sort(np.angle(np.linalg.eigvals(pars_true['A']))), 'k')
plt.plot(np.sort(np.angle(np.linalg.eigvals(pars_est_g['A']))), 'b')
plt.plot(np.sort(np.angle(np.linalg.eigvals(pars_est['A']))), 'g')
plt.show()

In [ ]:
plt.plot(np.real(ev_true), abs(np.imag(ev_true)), 'ko')
plt.plot(np.real(ev_est), np.imag(ev_est), 'go')
plt.plot(np.real(ev_est_g), np.imag(ev_est_g), 'bo')
plt.show()

# Fig 3 b - two subpop stitching as function of overlap

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
from scipy import spatial
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import calc_subspace_proj_error

from matplotlib import cm


rnd_seed = 31



data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e3/seed_'+str(rnd_seed)+'/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'

mmap, verbose = True, True

p,T_full,n,snr = 1000, 200020, 20, (1., 1.)
overlaps = (0,1,10,15,20,25,50,100,300,500,700,900,1000)
#overlaps = (0,10,15,20,25,50,100,500,900,1000)
#overlaps = (0,)

run = '_e3_init'
file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) +  run
load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
pars_true = load_file['pars_true'].copy()

ev_true = np.linalg.eigvals(pars_true['A'])
#ev_idx = np.argsort(np.angle(ev_true))
#ev_true = ev_true[n//2:]
std_angl = np.std(np.angle(ev_true))
std_tmsc = np.std(np.log(1-np.abs(ev_true)))    

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

clrs = cm.jet(np.linspace(50, 200, len(overlaps), dtype=int))
lgnd = []

plt.figure(figsize=(10,4))

subsp_errors   = np.zeros(len(overlaps))
dyn_errors_abs = np.zeros(len(overlaps))
dyn_errors_agl = np.zeros(len(overlaps))

subsp_errors_g   = np.zeros(len(overlaps))
dyn_errors_abs_g = np.zeros(len(overlaps))
dyn_errors_agl_g = np.zeros(len(overlaps))

run = '_e3'
for i in range(len(overlaps)):
    overlap = overlaps[i]
    
    print('overlap = ', str(overlap))
    
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T_full)+str(run)+'_'+str(overlap)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()
    obs_scheme = load_file['obs_scheme']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    traces = load_file['traces']
    traces_g = load_file['traces_g']
    W = load_file['W']
    Qs = load_file['Qs']
    Om = load_file['Om']
    
    C = pars_est['C'].copy()
    C[obs_scheme.sub_pops[0]] *= -1
    if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est['C']):
        pars_est['C'][obs_scheme.sub_pops[0]] *= -1
    del C    
    subsp_errors[i] = calc_subspace_proj_error(pars_true['C'], pars_est['C'])
    
    
    C = pars_est_g['C'].copy()
    C[obs_scheme.sub_pops[0]] *= -1
    if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est_g['C']):
        pars_est_g['C'][obs_scheme.sub_pops[0]] *= -1
    del C        
    subsp_errors_g[i] = calc_subspace_proj_error(pars_true['C'], pars_est_g['C'])
    
   
    plt.subplot(1,2,2)
    
    
    # ssidid
    ev_est = np.linalg.eigvals(pars_est['A'])
    #ev_idx = np.argsort(np.angle(ev_est))
    #ev_est = ev_est[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true)/std_angl, np.log(1-np.abs(ev_true))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est )/std_angl, np.log(1-np.abs(ev_est ))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)                
    #ev_true = ev_true[out[:,0]]
    #ev_est = ev_est[out[:,1]]            
    #dyn_errors_abs[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl[Ti] = np.sqrt(np.mean( (np.angle(ev_est) - np.angle(ev_true))**2))/np.pi * 90    
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs[i] = kde.mean() 
    dyn_errors_agl[i] = np.sum(np.log(kde))    

    # GROUSE
    ev_est_g = np.linalg.eigvals(pars_est_g['A'])
    #ev_idx = np.argsort(np.angle(ev_est_g))
    #ev_est_g = ev_est_g[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true )/std_angl, np.log(1-np.abs(ev_true ))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est_g)/std_angl, np.log(1-np.abs(ev_est_g))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)            
    #ev_true = ev_true[out[:,0]]
    #ev_est_g = ev_est_g[out[:,1]]            
    #dyn_errors_abs_g[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est_g))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl_g[Ti] = np.sqrt(np.mean( (np.angle(ev_est_g) - np.angle(ev_true))**2))/np.pi * 90
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs_g[i] = kde.mean() 
    dyn_errors_agl_g[i] = np.sum(np.log(kde))    
    
    """
    ev_est = np.linalg.eigvals(pars_est['A'])
    ev_idx = np.argsort(np.angle(ev_est))
    ev_est = ev_est[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true)/std_angl, np.log(1-np.abs(ev_true))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est )/std_angl, np.log(1-np.abs(ev_est ))/std_tmsc]).T
    out = find_nearest_pairs(x=x_,y=y_)        
    
    ev_true = ev_true[out[:,0]]
    ev_est = ev_est[out[:,1]]    
        
    dyn_errors_abs[i] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est))-np.log10(1-np.abs(ev_true)))**2 ))
    dyn_errors_agl[i] = np.sqrt(np.mean( (np.angle(ev_est) - np.angle(ev_true))**2))/np.pi * 90
    """

    """
    ev_est_g = np.linalg.eigvals(pars_est_g['A'])
    ev_idx = np.argsort(np.angle(ev_est_g))
    ev_est_g = ev_est_g[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true )/std_angl, np.log(1-np.abs(ev_true ))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est_g)/std_angl, np.log(1-np.abs(ev_est_g))/std_tmsc]).T
    out = find_nearest_pairs(x=x_,y=y_)        
    
    ev_true = ev_true[out[:,0]]
    ev_est_g = ev_est_g[out[:,1]]    
    
    dyn_errors_abs_g[i] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est_g))-np.log10(1-np.abs(ev_true)))**2 ))
    dyn_errors_agl_g[i] = np.sqrt(np.mean( (np.angle(ev_est_g) - np.angle(ev_true))**2))/np.pi * 90
    """
    
        
plt.subplot(1,2,1)
plt.loglog(overlaps, subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.loglog(overlaps, subsp_errors, 'o-', color='g', linewidth=2)
plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('overlap')
plt.title('subspace estimation')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))

plt.subplot(1,2,2)
    
plt.semilogx(overlaps, dyn_errors_agl_g, 'bo-', linewidth=1.5)
plt.semilogx(overlaps, dyn_errors_agl, 'go-', linewidth=2)
#plt.loglog(overlaps, dyn_errors_abs_g, 'b.-')
#plt.loglog(overlaps, dyn_errors_agl_g, 'bx--')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.box('off')
plt.ylabel('eigenvalues of A')
plt.xlabel('overlap')
plt.title('spectrum of dynamics')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-3,1))

plt.savefig(fig_path + 'fig3_B.pdf')
plt.show()
    
    

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
from scipy import spatial
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import calc_subspace_proj_error

from matplotlib import cm


rnd_seed = 3


data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e3/n20/seed_'+str(rnd_seed)+'/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'

mmap, verbose = True, True

p,T_full,n,snr = 1000, 200020, 20, (1., 1.)
overlaps = (0,1,10,15,20,25,50,100,300,500,700,900,1000)
overlaps = (0,10,15,20,25,50,100,500,900,1000)
#overlaps = (0,)

run = '_e3_init'
file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) +  run
load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
pars_true = load_file['pars_true'].copy()

ev_true = np.linalg.eigvals(pars_true['A'])
#ev_idx = np.argsort(np.angle(ev_true))
#ev_true = ev_true[n//2:]
std_angl = np.std(np.angle(ev_true))
std_tmsc = np.std(np.log(1-np.abs(ev_true)))    

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

clrs = cm.jet(np.linspace(50, 200, len(overlaps), dtype=int))
lgnd = []

plt.figure(figsize=(10,4))

subsp_errors   = np.zeros(len(overlaps))
subsp_errors_1 = np.zeros(len(overlaps))
subsp_errors_2 = np.zeros(len(overlaps))
dyn_errors_abs = np.zeros(len(overlaps))
dyn_errors_agl = np.zeros(len(overlaps))

subsp_errors_g   = np.zeros(len(overlaps))
dyn_errors_abs_g = np.zeros(len(overlaps))
dyn_errors_agl_g = np.zeros(len(overlaps))
subsp_errors_1_g = np.zeros(len(overlaps))
subsp_errors_2_g = np.zeros(len(overlaps))

run = '_e3'
for i in range(len(overlaps)):
    overlap = overlaps[i]
    
    print('overlap = ', str(overlap))
    
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T_full)+str(run)+'_'+str(overlap)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()
    obs_scheme = load_file['obs_scheme']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    traces = load_file['traces']
    traces_g = load_file['traces_g']
    W = load_file['W']
    Qs = load_file['Qs']
    Om = load_file['Om']
    
    sub_pops = obs_scheme.sub_pops
    C = pars_est['C'].copy()
    C[sub_pops[0],:] *= -1
    if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est['C']):
        pars_est['C'][sub_pops[0],:] *= -1
    del C    
    subsp_errors[i] = calc_subspace_proj_error(pars_true['C'], pars_est['C'])
    subsp_errors_1[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[0],:], pars_est['C'][sub_pops[0],:])
    subsp_errors_2[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[1],:], pars_est['C'][sub_pops[1],:])
    
    
    C = pars_est_g['C'].copy()
    C[obs_scheme.sub_pops[0]] *= -1
    if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est_g['C']):
        pars_est_g['C'][obs_scheme.sub_pops[0]] *= -1
    del C        
    subsp_errors_g[i] = calc_subspace_proj_error(pars_true['C'], pars_est_g['C'])
    subsp_errors_1_g[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[0],:], pars_est_g['C'][sub_pops[0],:])
    subsp_errors_2_g[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[1],:], pars_est_g['C'][sub_pops[1],:])
    
   
    """
    # ssidid
    ev_est = np.linalg.eigvals(pars_est['A'])
    #ev_idx = np.argsort(np.angle(ev_est))
    #ev_est = ev_est[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true)/std_angl, np.log(1-np.abs(ev_true))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est )/std_angl, np.log(1-np.abs(ev_est ))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)                
    #ev_true = ev_true[out[:,0]]
    #ev_est = ev_est[out[:,1]]            
    #dyn_errors_abs[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl[Ti] = np.sqrt(np.mean( (np.angle(ev_est) - np.angle(ev_true))**2))/np.pi * 90    
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs[i] = kde.mean() 
    dyn_errors_agl[i] = np.sum(np.log(kde))    

    # GROUSE
    ev_est_g = np.linalg.eigvals(pars_est_g['A'])
    #ev_idx = np.argsort(np.angle(ev_est_g))
    #ev_est_g = ev_est_g[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true )/std_angl, np.log(1-np.abs(ev_true ))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est_g)/std_angl, np.log(1-np.abs(ev_est_g))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)            
    #ev_true = ev_true[out[:,0]]
    #ev_est_g = ev_est_g[out[:,1]]            
    #dyn_errors_abs_g[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est_g))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl_g[Ti] = np.sqrt(np.mean( (np.angle(ev_est_g) - np.angle(ev_true))**2))/np.pi * 90
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs_g[i] = kde.mean() 
    dyn_errors_agl_g[i] = np.sum(np.log(kde))    
    
    """
        
plt.subplot(1,2,1)
plt.loglog(overlaps, subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.loglog(overlaps, subsp_errors, 'o-', color='g', linewidth=2)
plt.loglog(overlaps, subsp_errors_1, '.--', color='g', linewidth=2)
plt.loglog(overlaps, subsp_errors_2, '.--', color='g', linewidth=2)
plt.loglog(overlaps, subsp_errors_1_g, '.--', color='b', linewidth=1.5)
plt.loglog(overlaps, subsp_errors_2_g, '.--', color='b', linewidth=1.5)

plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('overlap')
plt.title('subspace estimation (log-log)')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))

plt.subplot(1,2,2)
plt.semilogy(overlaps, subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.semilogy(overlaps, subsp_errors, 'o-', color='g', linewidth=2)
plt.semilogy(overlaps, subsp_errors_1, '.--', color='g', linewidth=2)
plt.semilogy(overlaps, subsp_errors_2, '.--', color='g', linewidth=2)
plt.semilogy(overlaps, subsp_errors_1_g, '.--', color='b', linewidth=1.5)
plt.semilogy(overlaps, subsp_errors_2_g, '.--', color='b', linewidth=1.5)

plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('overlap')
plt.title('subspace estimation (semilogy)')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))

"""
plt.subplot(1,2,2)
    
plt.semilogx(overlaps, dyn_errors_agl_g, 'bo-', linewidth=1.5)
plt.semilogx(overlaps, dyn_errors_agl, 'go-', linewidth=2)
#plt.loglog(overlaps, dyn_errors_abs_g, 'b.-')
#plt.loglog(overlaps, dyn_errors_agl_g, 'bx--')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.box('off')
plt.ylabel('eigenvalues of A')
plt.xlabel('overlap')
plt.title('spectrum of dynamics')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-3,1))

"""    
plt.savefig(fig_path + 'fig3_B.pdf')
plt.show()
    

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import linalg as la
from scipy import spatial
import glob, os, psutil, time

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from subtracking import calc_subspace_proj_error

from matplotlib import cm

fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'

mmap, verbose = True, True

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)


rnd_seeds = range(30,40)
overlap = overlaps[0]

p,n,T_full = 1000, 10, 100030

plt.figure(figsize=(10,4))
clrs = cm.jet(np.linspace(50, 200, len(overlaps), dtype=int))
lgnd = []

subsp_errors   = np.zeros(len(rnd_seeds))
subsp_errors_1 = np.zeros(len(rnd_seeds))
subsp_errors_2 = np.zeros(len(rnd_seeds))

subsp_errors_g   = np.zeros(len(rnd_seeds))
subsp_errors_1_g = np.zeros(len(rnd_seeds))
subsp_errors_2_g = np.zeros(len(rnd_seeds))

for i in range(len(rnd_seeds)):
   
    rnd_seed = rnd_seeds[i]
    
    print('rnd_seed : ', str(rnd_seed))
    
    print('overlap = ', str(overlap))
    
    
    data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e3/seed_'+str(rnd_seed)+'/'

    run = '_e3_init'
    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T_full) +  run
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    pars_true = load_file['pars_true'].copy()

    ev_true = np.linalg.eigvals(pars_true['A'])
    #ev_idx = np.argsort(np.angle(ev_true))
    #ev_true = ev_true[n//2:]
    std_angl = np.std(np.angle(ev_true))
    std_tmsc = np.std(np.log(1-np.abs(ev_true)))    


    run = '_e3'
    file_name = 'p'+str(p)+'n'+str(n)+'T'+str(T_full)+str(run)+'_'+str(overlap)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()
    idx_a, idx_b = load_file['idx_a'].copy(), load_file['idx_b'].copy()
    obs_scheme = load_file['obs_scheme']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    traces = load_file['traces']
    traces_g = load_file['traces_g']
    W = load_file['W']
    Qs = load_file['Qs']
    Om = load_file['Om']
    
    sub_pops = obs_scheme.sub_pops
    C = pars_est['C'].copy()
    C[sub_pops[0],:] *= -1
    if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est['C']):
        pars_est['C'][sub_pops[0],:] *= -1
    del C    
    subsp_errors[i] = calc_subspace_proj_error(pars_true['C'], pars_est['C'])
    subsp_errors_1[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[0],:], pars_est['C'][sub_pops[0],:])
    subsp_errors_2[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[1],:], pars_est['C'][sub_pops[1],:])
    
    
    C = pars_est_g['C'].copy()
    C[obs_scheme.sub_pops[0]] *= -1
    if calc_subspace_proj_error(pars_true['C'], C) < calc_subspace_proj_error(pars_true['C'], pars_est_g['C']):
        pars_est_g['C'][obs_scheme.sub_pops[0]] *= -1
    del C        
    subsp_errors_g[i] = calc_subspace_proj_error(pars_true['C'], pars_est_g['C'])
    subsp_errors_1_g[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[0],:], pars_est_g['C'][sub_pops[0],:])
    subsp_errors_2_g[i] = calc_subspace_proj_error(pars_true['C'][sub_pops[1],:], pars_est_g['C'][sub_pops[1],:])
    
   
    plt.subplot(1,2,2)
    plt.plot((i+1)*np.ones(n), principal_angle(pars_true['C'], pars_est['C']), 'o', color='b', linewidth=1.5)
    plt.plot((i+1.2)*np.ones(n), principal_angle(pars_true['C'], pars_est_g['C']), 'o', color='g', linewidth=2)


    """
    # ssidid
    ev_est = np.linalg.eigvals(pars_est['A'])
    #ev_idx = np.argsort(np.angle(ev_est))
    #ev_est = ev_est[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true)/std_angl, np.log(1-np.abs(ev_true))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est )/std_angl, np.log(1-np.abs(ev_est ))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)                
    #ev_true = ev_true[out[:,0]]
    #ev_est = ev_est[out[:,1]]            
    #dyn_errors_abs[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl[Ti] = np.sqrt(np.mean( (np.angle(ev_est) - np.angle(ev_true))**2))/np.pi * 90    
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs[i] = kde.mean() 
    dyn_errors_agl[i] = np.sum(np.log(kde))    

    # GROUSE
    ev_est_g = np.linalg.eigvals(pars_est_g['A'])
    #ev_idx = np.argsort(np.angle(ev_est_g))
    #ev_est_g = ev_est_g[ev_idx][n//2:]        
    
    x_ = np.vstack([np.angle(ev_true )/std_angl, np.log(1-np.abs(ev_true ))/std_tmsc]).T
    y_ = np.vstack([np.angle(ev_est_g)/std_angl, np.log(1-np.abs(ev_est_g))/std_tmsc]).T
    
    #out = find_nearest_pairs(x=x_,y=y_)            
    #ev_true = ev_true[out[:,0]]
    #ev_est_g = ev_est_g[out[:,1]]            
    #dyn_errors_abs_g[Ti] = np.sqrt(np.mean( (np.log10(1-np.abs(ev_est_g))-np.log10(1-np.abs(ev_true)))**2 ))
    #dyn_errors_agl_g[Ti] = np.sqrt(np.mean( (np.angle(ev_est_g) - np.angle(ev_true))**2))/np.pi * 90
    
    kde = sp.stats.gaussian_kde(x_.T)(y_.T)
    dyn_errors_abs_g[i] = kde.mean() 
    dyn_errors_agl_g[i] = np.sum(np.log(kde))    
    
    """
        
plt.subplot(1,2,1)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors, 'o-', color='g', linewidth=2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_1, '.--', color='g', linewidth=2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_2, '.--', color='g', linewidth=2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_1_g, '.--', color='b', linewidth=1.5)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_2_g, '.--', color='b', linewidth=1.5)


plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('# system draw')
plt.title('subspace estimation')
plt.axis([0.5, len(rnd_seeds)+0.5, 0.01, 1])
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))

plt.subplot(1,2,2)

plt.box('off')
lgnd = ['GROUSE', 'ssidid']
#plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('principal angles')
plt.xlabel('# system draw')
plt.title('subspace estimation')
plt.axis([0.5, len(rnd_seeds)+0.5, 0.01, 1])
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))


plt.savefig(fig_path + 'fig3_Z.pdf')
plt.show()
    

In [ ]:
plt.subplot(1,2,1)
plt.loglog(range(1,len(rnd_seeds)+1), subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.loglog(range(1,len(rnd_seeds)+1), subsp_errors, 'o-', color='g', linewidth=2)
plt.loglog(range(1,len(rnd_seeds)+1), subsp_errors_1, '.--', color='g', linewidth=2)
plt.loglog(range(1,len(rnd_seeds)+1), subsp_errors_2, '.--', color='g', linewidth=2)
plt.loglog(range(1,len(rnd_seeds)+1), subsp_errors_1_g, '.--', color='b', linewidth=1.5)
plt.loglog(range(1,len(rnd_seeds)+1), subsp_errors_2_g, '.--', color='b', linewidth=1.5)

plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('overlap')
plt.title('subspace estimation (log-log)')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))

plt.subplot(1,2,2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_g, 'o-', color='b', linewidth=1.5)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors, 'o-', color='g', linewidth=2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_1, '.--', color='g', linewidth=2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_2, '.--', color='g', linewidth=2)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_1_g, '.--', color='b', linewidth=1.5)
plt.semilogy(range(1,len(rnd_seeds)+1), subsp_errors_2_g, '.--', color='b', linewidth=1.5)

plt.box('off')
lgnd = ['GROUSE', 'ssidid']
plt.legend(lgnd, loc=1,frameon=False)
plt.ylabel('subsp. proj. error')
plt.xlabel('overlap')
plt.title('subspace estimation (semilogy)')
#plt.xticks(10**np.arange(3,6))
#plt.yticks(10**np.arange(-2,1))


In [ ]:
ev_true = np.linalg.eigvals(pars_true['A'])

# Fig 4a

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from ssidid import ObservationScheme

fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'


p = 41
T = 100


# create subpopulations

# splitting imaging planes, 2 subpops
ns = 10

sub_pops = [np.arange(i*(p//ns), (i+1)*(p//ns)) for i in range(ns-1)] + [np.arange((ns-1)*(p//ns),p)]

obs_sweeps = 10

obs_pops = np.hstack([np.arange(len(sub_pops)) for i in range(obs_sweeps)])
obs_time = np.array([i*T//len(obs_pops) for i in range(1,len(obs_pops)+1)])


obs_scheme = ObservationScheme(p=p,T=T,
                               sub_pops=sub_pops, 
                               obs_pops=obs_pops,
                               obs_time=obs_time)
obs_scheme.gen_mask_from_scheme()

plt.figure(figsize=(16,10))
plt.subplot(3,2,1)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
#    plt.plot(10*i+x, 100-10*x, '--', color='b')
plt.xticks(range(10,101,10), range(1,11))
plt.yticks(np.arange(0,41,10)+.5, np.arange(1, 42, 10)[::-1])
plt.axis([0,100,0,p])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,1).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,1).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,2,3)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
    plt.plot(10*i+x, p-p*x/10, '--', color='b')
plt.xticks(range(10,101,10), range(1,11))
plt.yticks(np.arange(0,41,10)+.5, np.arange(1, 42, 10)[::-1])
plt.axis([0,100,0,p])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,3).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,3).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,2,5)
plt.imshow(obs_scheme.mask.T, interpolation='None', aspect='auto')
plt.gray()
plt.xticks(np.arange(10,101,10)-0.5, range(1,11,1))
plt.yticks(np.arange(0,41,10), np.arange(1, 42, 10))
plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,5).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,5).get_xaxis().set_tick_params(direction='out')

lag_range=np.arange(10)
W = obs_scheme.comp_coocurrence_weights(lag_range, sso=True)

for i in range(5):
    plt.subplot(6,4,3+i*4)
    plt.imshow(1/W[2*i], interpolation='None')
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('m = ' + str(2*i))
    plt.gray()

    plt.subplot(6,4,4+i*4)
    plt.imshow(1/W[2*i+1], interpolation='None')
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('m = ' + str(2*i+1))
    plt.gray()
    #if i == 4:
    #    plt.colorbar()
plt.subplot(6, 6, 35)    
plt.imshow(np.ones((10,10)))
plt.clim([0,1])
plt.xticks([])
plt.yticks([])
plt.ylabel('U m')


plt.savefig(fig_path + 'fig4_A.pdf')

plt.show()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from ssidid import ObservationScheme

fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'




p = 41
T = 1200
lag_range = np.arange(10)

# create subpopulations

# splitting imaging planes, 2 subpops
ns = 10

sub_pops = [np.arange(i*(p//ns), (i+1)*(p//ns)) for i in range(ns-1)] + [np.arange((ns-1)*(p//ns),p)]

obs_sweeps = 60

obs_pops = np.hstack([np.hstack((np.arange(0,len(sub_pops)),np.arange(len(sub_pops))[::-1])) for i in range(obs_sweeps)])
#obs_pops = np.hstack([np.arange(1), obs_pops, np.arange(1,len(sub_pops))])
obs_time = np.arange(1,T+1)
#obs_time = np.array([i*T//len(obs_pops) for i in range(1,len(obs_pops)+1)])

obs_scheme = ObservationScheme(p=p,T=T,
                               sub_pops=sub_pops, 
                               obs_pops=obs_pops,
                               obs_time=obs_time)
obs_scheme.gen_mask_from_scheme()

plt.figure(figsize=(16,10))
plt.subplot(3,2,1)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
#    plt.plot(10*i+x, 100-10*x, '--', color='b')
plt.xticks(range(10,101,10), range(1,11))
plt.yticks(np.arange(0,41,10)+.5, np.arange(1, 42, 10)[::-1])
plt.axis([0,100,0,p])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,1).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,1).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,2,3)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
    #plt.plot(10*i+x, p-p*x/10, '--', color='b')
    if np.mod(i,2)==0:
        plt.plot(10*i+x, p-p*x/10, '--', color='b')
    else:
        plt.plot(10*i+x, p*x/10, '--', color='b')
    
    
plt.xticks(range(10,101,10), range(1,11))
plt.yticks(np.arange(0,41,10)+.5, np.arange(1, 42, 10)[::-1])
plt.axis([0,100,0,p])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,3).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,3).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,2,5)
plt.imshow(obs_scheme.mask[:100,:].T, interpolation='None', aspect='auto')
plt.gray()
plt.xticks(np.arange(10,101,10)-0.5, range(1,11,1))
plt.yticks(np.arange(0,41,10), np.arange(1, 42, 10))
plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,5).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,5).get_xaxis().set_tick_params(direction='out')

lag_range=np.arange(10)
W = obs_scheme.comp_coocurrence_weights(lag_range, sso=True)

for i in range(5):
    plt.subplot(6,4,3+i*4)
    plt.imshow(1/W[2*i], interpolation='None')
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('m = ' + str(2*i))
    plt.gray()
    plt.clim(0,185)

    plt.subplot(6,4,4+i*4)
    plt.imshow(1/W[2*i+1], interpolation='None')
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('m = ' + str(2*i+1))
    plt.gray()
    plt.clim(0,185)
    #if i == 4:
    #    plt.colorbar()
plt.subplot(6, 6, 35)    
tmp = np.zeros_like(W[0])
for i in range(len(W)):
    tmp += 1/W[i]
plt.imshow(tmp, interpolation='None')  
plt.clim(0,185)
#plt.clim([0,1])
plt.xticks([])
plt.yticks([])
plt.ylabel('U m')


plt.savefig(fig_path + 'fig4_A_alt10.pdf')

plt.show()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from ssidid import ObservationScheme

fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'




p = 41
T = 10000
lag_range = np.arange(11)

# create subpopulations

# splitting imaging planes, 2 subpops
ns = 11

sub_pops = [np.arange(i*(4), (i+1)*(4)) for i in range(ns-1)] + [np.arange((ns-1)*4,p)]

obs_sweeps = 500

obs_pops = np.hstack([np.hstack((np.arange(1,len(sub_pops)),np.arange(len(sub_pops)-1)[::-1])) for i in range(obs_sweeps)])
#obs_pops = np.hstack([np.arange(1), obs_pops, np.arange(1,len(sub_pops))])
obs_time = np.arange(1,T+1)
#obs_time = np.array([i*T//len(obs_pops) for i in range(1,len(obs_pops)+1)])

obs_scheme = ObservationScheme(p=p,T=T,
                               sub_pops=sub_pops, 
                               obs_pops=obs_pops,
                               obs_time=obs_time)
obs_scheme.gen_mask_from_scheme()

plt.figure(figsize=(16,10))
plt.subplot(3,2,1)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
#    plt.plot(10*i+x, 100-10*x, '--', color='b')
plt.xticks(range(10,101,10), range(1,11))
plt.yticks(np.arange(0,41,10)+.5, np.arange(1, 42, 10)[::-1])
plt.axis([0,100,0,p])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,1).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,1).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,2,3)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
    #plt.plot(10*i+x, p-p*x/10, '--', color='b')
    if np.mod(i,2)==0:
        plt.plot(10*i+x, p-p*x/10, '--', color='b')
    else:
        plt.plot(10*i+x, p*x/10, '--', color='b')
    
    
plt.xticks(range(10,101,10), range(1,11))
plt.yticks(np.arange(0,41,10)+.5, np.arange(1, 42, 10)[::-1])
plt.axis([0,100,0,p])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,3).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,3).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,2,5)
plt.imshow(obs_scheme.mask[:100,:].T, interpolation='None', aspect='auto')
plt.gray()
plt.xticks(np.arange(10,101,10)-0.5, range(1,11,1))
plt.yticks(np.arange(0,41,10), np.arange(1, 42, 10))
plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,2,5).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,2,5).get_xaxis().set_tick_params(direction='out')

W = obs_scheme.comp_coocurrence_weights(lag_range, sso=True)

for i in range(5):
    plt.subplot(6,4,3+i*4)
    plt.imshow(1/W[2*i], interpolation='None')
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('m = ' + str(2*i))
    plt.gray()

    plt.subplot(6,4,4+i*4)
    plt.imshow(1/W[2*i+1], interpolation='None')
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('m = ' + str(2*i+1))
    plt.gray()
    #if i == 4:
    #    plt.colorbar()
plt.subplot(6,4,23)
plt.imshow(1/W[10], interpolation='None')
plt.xticks([])
plt.yticks([])
plt.ylabel('m = ' + str(11))
plt.gray()
    
plt.subplot(6,4,24)    
tmp = np.zeros_like(W[0])
for i in range(len(W)):
    tmp += 1/W[i]
plt.imshow(tmp, interpolation='None')
#plt.colorbar()
plt.clim([0,tmp.max()])
plt.xticks([])
plt.yticks([])
plt.ylabel('U m')


plt.savefig(fig_path + 'fig4_A_alt11.pdf')

plt.show()